In [1]:
import numpy as np
import tensorflow as tf
from keras import preprocessing
from keras.datasets import cifar10
from keras.layers import Dense, Dropout, Activation, Flatten, Lambda, BatchNormalization, merge, Conv2D
from keras.layers.merge import concatenate
from keras.layers import add
from keras.layers import Convolution2D, MaxPooling2D, AveragePooling2D
from keras.engine import Input, Model
from keras.optimizers import SGD
from keras.callbacks import Callback, LearningRateScheduler, ModelCheckpoint, EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
import keras.backend as K
import json
import time
import math, os, sys

from keras.utils.np_utils import to_categorical

Using TensorFlow backend.


In [2]:
nb_filters_reduction_factor = 8
nb_classes = 10

In [3]:
# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# reorder dimensions for tensorflow
x_train = np.transpose(x_train.astype('float32') / 255., (0, 1,2, 3))
x_test = np.transpose(x_test.astype('float32') / 255., (0,1, 2, 3))
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices

y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

# we reduce # filters by factor of 8 compared to original inception-v4

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [4]:
def inception_resnet_v2_stem(x):
    # in original inception-resnet-v2, conv stride is 2
    # x = Convolution2D(32 // nb_filters_reduction_factor, 3, 3, subsample=(1, 1), activation='relu',
    #                   init='he_normal', border_mode='valid', dim_ordering='tf')(x)
    x = Conv2D(32 // nb_filters_reduction_factor, (3, 3), activation="relu", strides=(1, 1), padding="valid", data_format="channels_last",
           kernel_initializer="he_normal")(x)

    # x = Convolution2D(32 // nb_filters_reduction_factor, 3, 3, subsample=(1, 1), activation='relu',
    #                   init='he_normal', border_mode='valid', dim_ordering='tf')(x)
    x = Conv2D(32 // nb_filters_reduction_factor, (3, 3), activation="relu", strides=(1, 1), padding="valid",
               data_format="channels_last",
               kernel_initializer="he_normal")(x)
    # x = Convolution2D(64 // nb_filters_reduction_factor, 3, 3, subsample=(1, 1), activation='relu',
    #                   init='he_normal', border_mode='same', dim_ordering='tf')(x)

    x = Conv2D(64 // nb_filters_reduction_factor, (3, 3), activation="relu", strides=(1, 1), padding="same",
               data_format="channels_last",
               kernel_initializer="he_normal")(x)

    # in original inception-resnet-v2, stride is 2
    a = MaxPooling2D((3, 3), strides=(1, 1), padding='valid',  data_format="channels_last")(x)
    # in original inception-resnet-v2, conv stride is 2
    # b = Convolution2D(96 // nb_filters_reduction_factor, 3, 3, subsample=(1, 1), activation='relu',
    #                   init='he_normal', border_mode='valid', dim_ordering='tf')(x)
    b = Conv2D(96 // nb_filters_reduction_factor, (3, 3), activation="relu", strides=(1, 1), padding="valid",
               data_format="channels_last",
               kernel_initializer="he_normal")(x)
    # x = merge([a, b], mode='concat', concat_axis=-1)
    x = concatenate([a, b], axis=-1)

    # a = Convolution2D(64 // nb_filters_reduction_factor, 1, 1, subsample=(1, 1), activation='relu',
    #                   init='he_normal', border_mode='same', dim_ordering='tf')(x)
    a = Conv2D(64 // nb_filters_reduction_factor, (3, 3), activation="relu", strides=(1, 1), padding="same",
               data_format="channels_last",
               kernel_initializer="he_normal")(x)
    # a = Convolution2D(96 // nb_filters_reduction_factor, 3, 3, subsample=(1, 1), activation='relu',
    #                   init='he_normal', border_mode='valid', dim_ordering='tf')(a)
    a = Conv2D(96 // nb_filters_reduction_factor, (3, 3), activation="relu", strides=(1, 1), padding="valid",
               data_format="channels_last",
               kernel_initializer="he_normal")(a)
    # b = Convolution2D(64 // nb_filters_reduction_factor, 1, 1, subsample=(1, 1), activation='relu',
    #                   init='he_normal', border_mode='same', dim_ordering='tf')(x)
    b = Conv2D(64 // nb_filters_reduction_factor, (1, 1), activation="relu", strides=(1, 1), padding="same",
               data_format="channels_last",
               kernel_initializer="he_normal")(x)
    # b = Convolution2D(64 // nb_filters_reduction_factor, 7, 1, subsample=(1, 1), activation='relu',
    #                   init='he_normal', border_mode='same', dim_ordering='tf')(b)

    b = Conv2D(64 // nb_filters_reduction_factor, (7, 1), activation="relu", strides=(1, 1), padding="same",
               data_format="channels_last",
               kernel_initializer="he_normal")(b)
    # b = Convolution2D(64 // nb_filters_reduction_factor, 1, 7, subsample=(1, 1), activation='relu',
    #                   init='he_normal', border_mode='same', dim_ordering='tf')(b)
    b = Conv2D(64 // nb_filters_reduction_factor, (1, 7), activation="relu", strides=(1, 1), padding="same",
               data_format="channels_last",
               kernel_initializer="he_normal")(b)
    # b = Convolution2D(96 // nb_filters_reduction_factor, 3, 3, subsample=(1, 1), activation='relu',
    #                   init='he_normal', border_mode='valid', dim_ordering='tf')(b)

    b = Conv2D(96 // nb_filters_reduction_factor, (3, 3), activation="relu", strides=(1, 1), padding="valid",
               data_format="channels_last",
               kernel_initializer="he_normal")(b)
    # x = merge([a, b], mode='concat', concat_axis=-1)
    x = concatenate([a, b], axis=-1)
    # in original inception-resnet-v2, conv stride should be 2
    # a = Convolution2D(192 // nb_filters_reduction_factor, 3, 3, subsample=(1, 1), activation='relu',
    #                   init='he_normal', border_mode='valid', dim_ordering='tf')(x)
    a = Conv2D(192 // nb_filters_reduction_factor, (3, 3), activation="relu", strides=(1, 1), padding="valid",
               data_format="channels_last",
               kernel_initializer="he_normal")(x)
    # in original inception-resnet-v2, stride is 2
    # b = MaxPooling2D((3, 3), strides=(1, 1), border_mode='valid', dim_ordering='tf')(x)

    b = MaxPooling2D((3, 3), strides=(1, 1), padding='valid', data_format="channels_last")(x)
    # x = merge([a, b], mode='concat', concat_axis=-1)
    x = concatenate([a, b], axis=-1)
    x = Activation('relu')(x)

    return x

In [5]:
def inception_resnet_v2_A(x):
    shortcut = x

    # a = Convolution2D(32 // nb_filters_reduction_factor, 1, 1, subsample=(1, 1), activation='relu',
    #                   init='he_normal', border_mode='same', dim_ordering='tf')(x)
    a = Conv2D(32 // nb_filters_reduction_factor, (1, 1), activation="relu", strides=(1, 1), padding="same",
               data_format="channels_last",
               kernel_initializer="he_normal")(x)
    # b = Convolution2D(32 // nb_filters_reduction_factor, 1, 1, subsample=(1, 1), activation='relu',
    #                   init='he_normal', border_mode='same', dim_ordering='tf')(x)
    b = Conv2D(32 // nb_filters_reduction_factor, (1, 1), activation="relu", strides=(1, 1), padding="same",
               data_format="channels_last",
               kernel_initializer="he_normal")(x)
    # b = Convolution2D(32 // nb_filters_reduction_factor, 3, 3, subsample=(1, 1), activation='relu',
    #                   init='he_normal', border_mode='same', dim_ordering='tf')(b)
    b = Conv2D(32 // nb_filters_reduction_factor, (3, 3), activation="relu", strides=(1, 1), padding="same",
               data_format="channels_last",
               kernel_initializer="he_normal")(b)
    # c = Convolution2D(32 // nb_filters_reduction_factor, 1, 1, subsample=(1, 1), activation='relu',
    #                   init='he_normal', border_mode='same', dim_ordering='tf')(x)
    c = Conv2D(32 // nb_filters_reduction_factor, (1, 1), activation="relu", strides=(1, 1), padding="same",
               data_format="channels_last",
               kernel_initializer="he_normal")(x)
    # c = Convolution2D(48 // nb_filters_reduction_factor, 3, 3, subsample=(1, 1), activation='relu',
    #                   init='he_normal', border_mode='same', dim_ordering='tf')(c)
    c = Conv2D(48 // nb_filters_reduction_factor, (3,3), activation="relu", strides=(1, 1), padding="same",
               data_format="channels_last",
               kernel_initializer="he_normal")(c)
    # c = Convolution2D(64 // nb_filters_reduction_factor, 3, 3, subsample=(1, 1), activation='relu',
    #                   init='he_normal', border_mode='same', dim_ordering='tf')(c)
    c = Conv2D(64 // nb_filters_reduction_factor, (3,3), activation="relu", strides=(1, 1), padding="same",
               data_format="channels_last",
               kernel_initializer="he_normal")(c)
    # x = merge([a, b, c], mode='concat', concat_axis=-1)
    x = concatenate([a, b,c], axis=-1)
    # x = Convolution2D(384 // nb_filters_reduction_factor, 1, 1, subsample=(1, 1), activation='linear',
    #                   init='he_normal', border_mode='same', dim_ordering='tf')(x)
    x = Conv2D(384 // nb_filters_reduction_factor, (1, 1), activation="linear", strides=(1, 1), padding="same",
               data_format="channels_last",
               kernel_initializer="he_normal")(x)
    # x = merge([shortcut, x], mode='sum')
    x = add([shortcut, x])
    x = Activation('relu')(x)

    return x

In [6]:
def inception_resnet_v2_reduction_A(x):
    # a = MaxPooling2D((3, 3), strides=(2, 2), border_mode='valid', dim_ordering='tf')(x)
    a = MaxPooling2D((3, 3), strides=(2, 2), padding='valid', data_format="channels_last")(x)
    # b = Convolution2D(384 // nb_filters_reduction_factor, 3, 3, subsample=(2, 2), activation='relu',
    #                   init='he_normal', border_mode='valid', dim_ordering='tf')(x)
    b = Conv2D(384 // nb_filters_reduction_factor, (3,3), activation="relu", strides=(2, 2), padding="valid",
               data_format="channels_last",
               kernel_initializer="he_normal")(x)
    # c = Convolution2D(256 // nb_filters_reduction_factor, 1, 1, subsample=(1, 1), activation='relu',
    #                   init='he_normal', border_mode='same', dim_ordering='tf')(x)
    c = Conv2D(256 // nb_filters_reduction_factor, (1, 1), activation="relu", strides=(1, 1), padding="same",
               data_format="channels_last",
               kernel_initializer="he_normal")(x)
    # c = Convolution2D(256 // nb_filters_reduction_factor, 3, 3, subsample=(1, 1), activation='relu',
    #                   init='he_normal', border_mode='same', dim_ordering='tf')(c)
    c = Conv2D(256 // nb_filters_reduction_factor, (3,3), activation="relu", strides=(1, 1), padding="same",
               data_format="channels_last",
               kernel_initializer="he_normal")(c)
    # c = Convolution2D(384 // nb_filters_reduction_factor, 3, 3, subsample=(2, 2), activation='relu',
    #                   init='he_normal', border_mode='valid', dim_ordering='tf')(c)
    c = Conv2D(384 // nb_filters_reduction_factor, (3, 3), activation="relu", strides=(2,2), padding="valid",
               data_format="channels_last",
               kernel_initializer="he_normal")(c)
    # x = merge([a, b, c], mode='concat', concat_axis=-1)
    x = concatenate([a, b, c], axis=-1)
    return x

In [7]:
def inception_resnet_v2_B(x):
    shortcut = x

    # a = Convolution2D(192 // nb_filters_reduction_factor, 1, 1, subsample=(1, 1), activation='relu',
    #                   init='he_normal', border_mode='same', dim_ordering='tf')(x)
    a = Conv2D(192 // nb_filters_reduction_factor, (1,1), activation="relu", strides=(1,1), padding="same",
               data_format="channels_last",
               kernel_initializer="he_normal")(x)
    # b = Convolution2D(128 // nb_filters_reduction_factor, 1, 1, subsample=(1, 1), activation='relu',
    #                   init='he_normal', border_mode='same', dim_ordering='tf')(x)
    b = Conv2D(128 // nb_filters_reduction_factor, (1, 1), activation="relu", strides=(1, 1), padding="same",
               data_format="channels_last",
               kernel_initializer="he_normal")(x)
    # b = Convolution2D(160 // nb_filters_reduction_factor, 1, 7, subsample=(1, 1), activation='relu',
    #                   init='he_normal', border_mode='same', dim_ordering='tf')(b)
    b = Conv2D(160 // nb_filters_reduction_factor, (1, 7), activation="relu", strides=(1, 1), padding="same",
               data_format="channels_last",
               kernel_initializer="he_normal")(b)
    # b = Convolution2D(192 // nb_filters_reduction_factor, 7, 1, subsample=(1, 1), activation='relu',
    #                   init='he_normal', border_mode='same', dim_ordering='tf')(b)
    b = Conv2D(192 // nb_filters_reduction_factor, (7, 1), activation="relu", strides=(1, 1), padding="same",
               data_format="channels_last",
               kernel_initializer="he_normal")(b)
    # x = merge([a, b], mode='concat', concat_axis=-1)
    x = concatenate([a, b], axis=-1)
    # x = Convolution2D(1154 // nb_filters_reduction_factor, 1, 1, subsample=(1, 1), activation='linear',
    #                   init='he_normal', border_mode='same', dim_ordering='tf')(x)
    x = Conv2D(1154 // nb_filters_reduction_factor, (1, 1), activation="linear", strides=(1, 1), padding="same",
               data_format="channels_last",
               kernel_initializer="he_normal")(x)
    # x = merge([shortcut, x], mode='sum')
    x = add([shortcut, x])
    x = Activation('relu')(x)

    return x

In [8]:
def inception_resnet_v2_reduction_B(x):
    # a = MaxPooling2D((3, 3), strides=(2, 2), border_mode='valid', dim_ordering='tf')(x)
    a = MaxPooling2D((3, 3), strides=(2, 2), padding='valid', data_format="channels_last")(x)
    # b = Convolution2D(256 // nb_filters_reduction_factor, 1, 1, subsample=(1, 1), activation='relu',
    #                   init='he_normal', border_mode='same', dim_ordering='tf')(x)
    b = Conv2D(256 // nb_filters_reduction_factor, (1, 1), activation="relu", strides=(1, 1), padding="same",
               data_format="channels_last",
               kernel_initializer="he_normal")(x)
    # b = Convolution2D(288 // nb_filters_reduction_factor, 3, 3, subsample=(2, 2), activation='relu',
    #                   init='he_normal', border_mode='valid', dim_ordering='tf')(b)
    b = Conv2D(288 // nb_filters_reduction_factor, (3,3), activation="relu", strides=(2,2), padding="valid",
               data_format="channels_last",
               kernel_initializer="he_normal")(b)
    # c = Convolution2D(256 // nb_filters_reduction_factor, 1, 1, subsample=(1, 1), activation='relu',
    #                   init='he_normal', border_mode='same', dim_ordering='tf')(x)
    c = Conv2D(256 // nb_filters_reduction_factor, (1,1), activation="relu", strides=(1,1), padding="same",
               data_format="channels_last",
               kernel_initializer="he_normal")(x)
    # c = Convolution2D(288 // nb_filters_reduction_factor, 3, 3, subsample=(2, 2), activation='relu',
    #                   init='he_normal', border_mode='valid', dim_ordering='tf')(c)
    c = Conv2D(288 // nb_filters_reduction_factor, (3,3), activation="relu", strides=(2, 2), padding="valid",
               data_format="channels_last",
               kernel_initializer="he_normal")(c)
    # d = Convolution2D(256 // nb_filters_reduction_factor, 1, 1, subsample=(1, 1), activation='relu',
    #                   init='he_normal', border_mode='same', dim_ordering='tf')(x)
    d = Conv2D(256 // nb_filters_reduction_factor, (1,1), activation="relu", strides=(1,1), padding="same",
               data_format="channels_last",
               kernel_initializer="he_normal")(x)
    # d = Convolution2D(288 // nb_filters_reduction_factor, 3, 3, subsample=(1, 1), activation='relu',
    #                   init='he_normal', border_mode='same', dim_ordering='tf')(d)
    d = Conv2D(288 // nb_filters_reduction_factor, (3,3), activation="relu", strides=(1, 1), padding="same",
               data_format="channels_last",
               kernel_initializer="he_normal")(d)
    # d = Convolution2D(320 // nb_filters_reduction_factor, 3, 3, subsample=(2, 2), activation='relu',
    #                   init='he_normal', border_mode='valid', dim_ordering='tf')(d)
    d = Conv2D(320 // nb_filters_reduction_factor, (3, 3), activation="relu", strides=(2,2), padding="valid",
               data_format="channels_last",
               kernel_initializer="he_normal")(d)
    # x = merge([a, b, c, d], mode='concat', concat_axis=-1)
    x = concatenate([a, b, c,d], axis=-1)

    return x

In [9]:
def inception_resnet_v2_C(x):
    shortcut = x

    # a = Convolution2D(192 // nb_filters_reduction_factor, 1, 1, subsample=(1, 1), activation='relu',
    #                   init='he_normal', border_mode='same', dim_ordering='tf')(x)
    a = Conv2D(192 // nb_filters_reduction_factor, (1,1), activation="relu", strides=(1, 1), padding="same",
               data_format="channels_last",
               kernel_initializer="he_normal")(x)
    # b = Convolution2D(192 // nb_filters_reduction_factor, 1, 1, subsample=(1, 1), activation='relu',init='he_normal', border_mode='same', dim_ordering='tf')(x)
    b = Conv2D(192 // nb_filters_reduction_factor, (1, 1), activation="relu", strides=(1, 1), padding="same",
               data_format="channels_last",
               kernel_initializer="he_normal")(x)
    # b = Convolution2D(224 // nb_filters_reduction_factor, 1, 3, subsample=(1, 1), activation='relu',
    #                   init='he_normal', border_mode='same', dim_ordering='tf')(b)
    b = Conv2D(224 // nb_filters_reduction_factor, (1, 3), activation="relu", strides=(1, 1), padding="same",
               data_format="channels_last",
               kernel_initializer="he_normal")(b)
    # b = Convolution2D(256 // nb_filters_reduction_factor, 3, 1, subsample=(1, 1), activation='relu',
    #                   init='he_normal', border_mode='same', dim_ordering='tf')(b)
    b = Conv2D(256 // nb_filters_reduction_factor, (3,1), activation="relu", strides=(1, 1), padding="same",
               data_format="channels_last",
               kernel_initializer="he_normal")(b)
    # x = merge([a, b], mode='concat', concat_axis=-1)
    x = concatenate([a, b], axis=-1)
    # x = Convolution2D(2048 // nb_filters_reduction_factor, 1, 1, subsample=(1, 1), activation='linear',
    #                   init='he_normal', border_mode='same', dim_ordering='tf')(x)
    x = Conv2D(2048 // nb_filters_reduction_factor, (1, 1), activation="linear", strides=(1, 1), padding="same",
               data_format="channels_last",
               kernel_initializer="he_normal")(x)
    # x = merge([shortcut, x], mode='sum')
    x = add([shortcut, x])
    x = Activation('relu')(x)

    return x

In [10]:
img_rows, img_cols = 32, 32
img_channels = 3

# in original inception-resnet-v2, these are 5, 10, 5, respectively
num_A_blocks = 1
num_B_blocks = 1
num_C_blocks = 1

In [11]:
inputs = Input(shape=(img_rows, img_cols, img_channels))

x = inception_resnet_v2_stem(inputs)
for i in range(num_A_blocks):
    x = inception_resnet_v2_A(x)
x = inception_resnet_v2_reduction_A(x)
for i in range(num_B_blocks):
    x = inception_resnet_v2_B(x)
x = inception_resnet_v2_reduction_B(x)
for i in range(num_C_blocks):
    x = inception_resnet_v2_C(x)

# x = AveragePooling2D(pool_size=(4, 4), strides=(1, 1), border_mode='valid', dim_ordering='tf')(x)
x = AveragePooling2D(pool_size=(4, 4), strides=(1, 1), padding="valid", data_format="channels_last")(x)
x = Dropout(0.5)(x)
x = Flatten()(x)

In [12]:
predictions = Dense(nb_classes, activation='softmax')(x)

model = Model(input=inputs, output=predictions)

# model = Model(inputs=tf.Tensor(input=inputs, output=predictions))
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 30, 30, 4)    112         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 28, 28, 4)    148         conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_3 (Conv2D)               (None, 28, 28, 8)    296         conv2d_2[0][0]                   
__________________________________________________________________________________________________
max_poolin

D:\Anaconda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  This is separate from the ipykernel package so we can avoid doing imports until


In [13]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
batch_size = 128
nb_epoch = 10

# Model saving callback
#checkpointer = ModelCheckpoint(filepath='stochastic_depth_cifar10.hdf5', verbose=1, save_best_only=True)

SIZE = (32, 32)
BATCH_SIZE = 32

history = model.fit(x_train, y_train,
                    batch_size=batch_size, nb_epoch=nb_epoch, verbose=2,
                    validation_data=(x_test, y_test))


D:\Anaconda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:15: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  from ipykernel import kernelapp as app


Train on 50000 samples, validate on 10000 samples
Epoch 1/10


KeyboardInterrupt: 

In [15]:
from keras import optimizers
from keras.callbacks import LearningRateScheduler, TensorBoard, ModelCheckpoint
log_filepath  = './inception_resnet_v2'

In [16]:
# set optimizer
sgd = optimizers.SGD(lr=.1, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

# set callback
tb_cb = TensorBoard(log_dir=log_filepath, histogram_freq=0)
change_lr = 0.01
cbks = [change_lr,tb_cb]

# set data augmentation
datagen = ImageDataGenerator(horizontal_flip=True,
                             width_shift_range=0.125,
                             height_shift_range=0.125,
                             fill_mode='constant',cval=0.)
datagen.fit(x_train)

# start training
model.fit_generator(datagen.flow(x_train, y_train,batch_size=batch_size),
                    steps_per_epoch=64,
                    epochs=2,
                    validation_data=(x_test, y_test),
                    verbose=2)

Epoch 1/2


KeyboardInterrupt: 